## Running scPROTEIN stage2 on SCoPE2_Specht dataset

In this tutorial, we show how to run scPROTEIN using protein-level data as input and generate the cell embedding.

In [1]:
import argparse
import random
import numpy as np 
import scipy.sparse as sp
import os
from sklearn import metrics
from sklearn.metrics import silhouette_score,adjusted_rand_score,normalized_mutual_info_score
from sklearn.metrics.cluster import contingency_matrix
from scprotein import *

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

parser = argparse.ArgumentParser()
parser.add_argument("--stage1", type=bool, default=True, help='if scPROTEIN starts from stage1')
parser.add_argument("--learning_rate", type=float, default=1e-3, help='learning rate')
parser.add_argument("--num_hidden", type=int, default=400, help='hidden dimension') 
parser.add_argument("--num_proj_hidden", type=int, default=256, help='dimension of projection head')
parser.add_argument("--activation", type=str, default='prelu', help='activation function') 
parser.add_argument("--num_layers", type=int, default=2, help='num of GCN layers')
parser.add_argument("--num_protos", type=int, default=2, help='num of prototypes')
parser.add_argument("--num_changed_edges", type=int, default=50, help='num of added/removed edges')
parser.add_argument("--topology_denoising", type=bool, default=False, help='if scPROTEIN uses topology denoising')
parser.add_argument("--drop_edge_rate_1", type=float, default=0.2, help='dropedge rate for view1')
parser.add_argument("--drop_edge_rate_2", type=float, default=0.4, help='dropedge rate for view2')
parser.add_argument("--drop_feature_rate_1", type=float, default=0.4, help='mask_feature rate for view1')
parser.add_argument("--drop_feature_rate_2", type=float, default=0.2, help='mask_feature rate for view2')
parser.add_argument("--alpha", type=float, default=0.05, help='balance factor')
parser.add_argument("--tau", type=float, default=0.4, help='temperature coefficient')
parser.add_argument("--weight_decay", type=float, default=0.00001, help='weight_decay')
parser.add_argument("--num_epochs", type=int, default=200, help='Number of epochs.')
parser.add_argument("--seed", type=int, default=39788, help='Random seed.') 
parser.add_argument("--threshold", type=float, default=0.15, help='threshold of graph construct')
parser.add_argument("--feature_preprocess", type=bool, default=True, help='feature preprocess')
args =parser.parse_known_args()[0]   
setup_seed(args.seed)
activation = nn.PReLU() if args.activation == 'prelu' else F.relu


### Load the protein-level data from stage1 and establish the stage2 framework.


The following functions are used for stage2 construction:
<br/>

**graph_generation(features, threshold, feature_preprocess)**

**- Function:**

This function constructs the cell graph based on the protein feature matrix.

**- Parameters:**

- `features` (array): Single-cell proteomics data matrix.
- `threshold` (float): Threshold for graph construction.
- `feature_preprocess` (bool): Feature preprocessing.

**- Returns:**

- `graph_data` (torch_geometric data object): The graph data in torch_geometric format, consisting of edges and node features.



<br/>


**Encoder(input_features, num_hidden, activation, num_layers)**

**- Function:**

Construct the graph encoder for embedding learning.

**- Parameters:**

- `input_features` (int): Dimension of the input feature matrix (usually the number of proteins).
- `num_hidden` (int): Hidden dimension in the graph encoder.
- `activation` (str): The type of non-linear activation function.
- `num_layers` (int): Number of layers in the graph encoder.

**- Returns:**

- `encoder` (PyTorch module): The defined graph encoder module.


<br/>


**Model(encoder, num_hidden, num_proj_hidden, tau)**

**- Function:**

This function establishes the scPROTEIN *stage2* model, consisting of a graph encoder, projection head, and loss calculation.

**- Parameters:**

- `encoder` (PyTorch module): Defined graph encoder.
- `num_hidden` (int): Hidden dimension in the graph encoder.
- `num_proj_hidden` (int): Hidden dimension of the projection head.
- `tau` (float): Temperature coefficient.

**- Returns:**

- `model` (PyTorch module): The defined scPROTEIN *stage2* model.



<br/>



**scPROTEIN_learning(model, device, data, drop_feature_rate_1, drop_feature_rate_2, drop_edge_rate_1, drop_edge_rate_2, learning_rate, weight_decay, num_protos, topology_denoising, num_epochs, alpha, num_changed_edges, seed)**

**- Function:**

This function constructs the framework of scPROTEIN *stage2* training and prediction.

**- Parameters:**

- `model` (PyTorch module): Defined scPROTEIN *stage2* model.
- `device` (str): Running device.
- `data` (torch_geometric data): The defined graph data in torch_geometric format, consisting of edges and node features.
- `drop_feature_rate_1` (float): Dropedge rate for augmentation view1.
- `drop_feature_rate_2` (float): Dropedge rate for augmentation view2.
- `drop_edge_rate_1` (float): Feature masking rate for augmentation view1.
- `drop_edge_rate_2` (float): Feature masking rate for augmentation view2.
- `learning_rate` (float): Learning rate for Adam optimizer.
- `weight_decay` (float): Weight decay for Adam optimizer.
- `num_protos` (int): Number of prototypes.
- `topology_denoising` (bool): Indicator of if using the topology denoising.
- `num_epochs` (int): Number of epochs for training *stage2*. We empirically set 200 to strike a balance between achieving convergence and reducing training time.
- `alpha` (float): Balance factor in the loss function.
- `num_changed_edges` (int): Number of added/removed edges in topology denoising.
- `seed` (int): Random seed.

**- Returns:**

- `scPROTEIN` object for *stage2*. The functions of `scPROTEIN` are as follows:

    - `scPROTEIN.train()`: Conduct training of scPROTEIN *stage2*.
    - `scPROTEIN.embedding_generation()`: Generate the cell representation matrix based on the trained *stage2* model.



In [3]:
protein_list, cell_list, features = load_sc_proteomic_features(args.stage1)  
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
data = graph_generation(features, args.threshold, args.feature_preprocess).to(device)
torch.cuda.empty_cache()
encoder = Encoder(data.num_features, args.num_hidden, activation, k=args.num_layers).to(device)
model = Model(encoder, args.num_hidden, args.num_proj_hidden, args.tau).to(device)
scPROTEIN = scPROTEIN_learning(model,device, data, args.drop_feature_rate_1,args.drop_feature_rate_2,args.drop_edge_rate_1,args.drop_edge_rate_2,
                 args.learning_rate, args.weight_decay, args.num_protos, args.topology_denoising, args.num_epochs, args.alpha, args.num_changed_edges,args.seed)

### Model training

In [4]:
scPROTEIN.train()

(T) | Epoch=001, loss=8.0031 
(T) | Epoch=002, loss=7.6771 
(T) | Epoch=003, loss=7.3166 
(T) | Epoch=004, loss=7.2599 
(T) | Epoch=005, loss=7.2623 
(T) | Epoch=006, loss=7.2776 
(T) | Epoch=007, loss=7.2702 
(T) | Epoch=008, loss=7.2547 
(T) | Epoch=009, loss=7.2358 
(T) | Epoch=010, loss=7.2018 
(T) | Epoch=011, loss=7.1948 
(T) | Epoch=012, loss=7.1154 
(T) | Epoch=013, loss=7.0538 
(T) | Epoch=014, loss=6.9621 
(T) | Epoch=015, loss=6.9185 
(T) | Epoch=016, loss=6.8925 
(T) | Epoch=017, loss=6.8923 
(T) | Epoch=018, loss=6.8674 
(T) | Epoch=019, loss=6.9867 
(T) | Epoch=020, loss=6.9094 
(T) | Epoch=021, loss=6.8531 
(T) | Epoch=022, loss=6.9533 
(T) | Epoch=023, loss=6.9475 
(T) | Epoch=024, loss=6.8666 
(T) | Epoch=025, loss=6.8541 
(T) | Epoch=026, loss=6.8498 
(T) | Epoch=027, loss=6.9048 
(T) | Epoch=028, loss=6.7885 
(T) | Epoch=029, loss=6.8036 
(T) | Epoch=030, loss=6.7530 
(T) | Epoch=031, loss=6.8465 
(T) | Epoch=032, loss=7.0136 
(T) | Epoch=033, loss=6.7679 
(T) | Epoc

### Generate the cell embedding

In [5]:
embedding = scPROTEIN.embedding_generation()
embedding.shape

(1490, 400)

### Evaluation

In [6]:
def purity_score(y_true, y_pred):
    contingency_matrix1 = contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix1, axis=0)) / np.sum(contingency_matrix1) 


def dimension_reduce(embedding):
    X_trans_PCA = PCA(n_components=50, random_state=seed).fit_transform(embedding)  
    X_trans = TSNE(n_components=2,random_state=seed).fit_transform(X_trans_PCA)
    return X_trans

seed = 1
Y_cell_type_label = load_cell_type_labels()
label_dict = {'sc_m0':0, 'sc_u':1}
target_names = ['Macrophage','Monocyte']
Y_label = np.array(itemgetter(*list(Y_cell_type_label))(label_dict))

k_means = KMeans(n_clusters=len(target_names))
y_predict = k_means.fit_predict(embedding)
df_result = pd.DataFrame()
df_result['ARI'] = [np.round(adjusted_rand_score(Y_label,y_predict),3)]
df_result['ASW'] = [np.round(silhouette_score(embedding,y_predict),3)]
df_result['NMI'] = [np.round(normalized_mutual_info_score(Y_label,y_predict),3)]
df_result['PS'] = [np.round(purity_score(Y_label,y_predict),3)]
print('cell clustering result:')
print(df_result)


cell clustering result:
     ARI    ASW    NMI     PS
0  0.435  0.469  0.428  0.831
